In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import math
import random

In [4]:
data = pd.read_csv('Bachelor/bachelor2020/data/data.csv', sep=',', header=0)
x_data = data.drop(['id', 'time', 'lat', 'lon'],axis=1) # drop lat lon, time
y_data = data[[ 'time', 'id', 'lat', 'lon']]

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)
X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

In [2]:
learn_data = pd.read_csv('my_data/data_learn.csv', sep=',', header=0)
learn_data.sample(frac=1)
x_data = learn_data.drop(['id', 'time', 'lat', 'lon'],axis=1) # drop lat lon, time
y_data = learn_data[[ 'time', 'id', 'lat', 'lon']]

test_data = pd.read_csv('my_data/data_test.csv', sep=',', header=0) # scale test data

x_test = test_data.drop(['id', 'time', 'lat', 'lon'],axis=1)
y_test = test_data[[ 'time', 'id', 'lat', 'lon']]

x_data = preprocessing.scale(x_data)

x_test = preprocessing.scale(x_test)


In [114]:
a = np.array(y_data['id'].drop_duplicates())
len(a)

165

In [35]:
## WkNN

In [3]:
classifier = KNeighborsClassifier(n_neighbors=4, metric='euclidean', weights='distance', p=1, algorithm='auto')
classifier.fit(x_data, y_data['id'])
classifier.score(x_test, y_test['id'])


0.1784776902887139

In [66]:
classifier.predict(x_test[0:1])

array([165.])

In [64]:
classifier.predict_proba(x_test)

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.88611968],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.65146286,
        0.34853714],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
## Custom evaluation method



In [4]:
distance_between_sectors = pd.read_csv('my_data/distance_between_sector.csv', sep=',', header=0)

distance = distance_between_sectors.loc[distance_between_sectors['id'] == 164, '165'].values[0]

In [76]:
distance_between_sectors.loc[distance_between_sectors['id'] == 165, str('{:.0f}'.format(164))].values[0]

1.58

In [96]:
len(classifier.predict_proba(x_test[1:2])[0])

164

In [6]:
last_prediction = 0
correct = 0

for row in classifier.predict_proba(x_test):
    sector_and_distance = [100, 100] # sector number and distance to last predicted sector
    for index, row_j in enumerate(row):
        
        if last_prediction == 0:
            last_prediction = classifier.predict(x_test[0:1])[0]
            
        
        if row_j > 0:
            distance = distance_between_sectors.loc[distance_between_sectors['id'] == last_prediction, str('{:.0f}'.format(index + 1))].values[0]
            
            
            if distance < sector_and_distance[1]:
                sector_and_distance[0] = index + 1
                sector_and_distance[1] = distance
    
    
    last_prediction = sector_and_distance[0]

    if last_prediction == y_test.iloc[[index]]['id'].values[0]:
        correct = correct + 1

accuracy = correct/len(x_test)
accuracy

0.015748031496062992

In [7]:
correct = 0

for index, row in enumerate(x_test):
    distance = distance_between_sectors.loc[distance_between_sectors['id'] == y_test.iloc[[index]]['id'].values[0], str('{:.0f}'.format(classifier.predict(x_test[[index]])[0]))].values[0]
    if distance == 0:
        correct = correct + 1
    elif distance >= 1 and distance < 2:
        correct = correct + 0.7
    elif distance >= 2 and distance < 3:
        correct = correct + 0.4
    elif distance >= 3 and distance < 4:
        correct = correct + 0.1
    
accuracy = correct/len(x_test)
accuracy

0.3832020997375327